# Q&A chatbot with history supported by IRIS, OpenAI and Langchain

This is an adaption of the [Langchain chatbot with history example](https://python.langchain.com/docs/use_cases/question_answering/chat_history/) changed to use IRIS as its vector database.

The link between Langchain and IRIS is done by [langchain-iris](https://github.com/caretdev/langchain-iris).

# Setup

In [1]:
%pip install --upgrade --quiet langchain langchain-community langchainhub bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -qU langchain-iris testcontainers-iris

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_iris import IRISVector
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

import getpass
import os

## OpenAI

In [4]:
%pip install -q langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [6]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

## Google

In [7]:
# %pip install -q langchain_google_vertexai

In [8]:
# os.environ["GOOGLE_API_KEY"] = getpass.getpass()

In [9]:
# from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

# llm = ChatVertexAI(model="gemini-pro")

# IRIS Connection

In [10]:
!docker pull intersystemsdc/iris-community:2024.1-preview

2024.1-preview: Pulling from intersystemsdc/iris-community
Digest: sha256:5ffc1adeaac16d945d308cdefa03263c61bf8aafa52d43551a57ca3e5286db33
Status: Image is up to date for intersystemsdc/iris-community:2024.1-preview
docker.io/intersystemsdc/iris-community:2024.1-preview



What's Next?
  View summary of image vulnerabilities and recommendations â†’ docker scout quickview intersystemsdc/iris-community:2024.1-preview


In [11]:
import time
import os
from testcontainers.iris import IRISContainer

# Enterprise version with iris.key placed in the user's home directory
# license_key = os.path.abspath(os.path.expanduser("~/iris.key"))
# image = 'containers.intersystems.com/intersystems/iris:2023.3'

# Community Edition
license_key = None
image = 'intersystemsdc/iris-community:2024.1-preview'

container = IRISContainer(image, username="demo", password="demo", namespace="demo", license_key=license_key)
container.with_exposed_ports(1972, 52773)
container.start()
CONNECTION_STRING = container.get_connection_url("localhost")

time.sleep(1)
print('Started', CONNECTION_STRING)
print('Portal: ', f"http://localhost:{container.get_exposed_port(52773)}/csp/sys/UtilHome.csp")

# Constructing the retriever

In [13]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

COLLECTION_NAME = "chatbot_docs"
vectorstore = IRISVector.from_documents(
    embedding=OpenAIEmbeddings(),
    documents=splits,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

retriever = vectorstore.as_retriever()

# Playing with the retriever

Fisrt, let's search for documents that best match the user query for task decomposition approaches.

In [14]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
for doc in retrieved_docs:
    print(doc)
    print('-'*80)

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
--------------------------------------------------------------------------------
page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps 

Now, let's do the same query semantically speaking but syntactic different, i.e., using different words and see what the Vector Search engine returns:

In [15]:
retrieved_docs = retriever.invoke("How to implement Task Decomposition?")
for doc in retrieved_docs:
    print(doc)
    print('-'*80)

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
--------------------------------------------------------------------------------
page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they 

You can note that the results are practically the same, even with passing different query strings. This means that the embedding vectors are somehow abstracting the semantics in documents.

To get more evidences of such a semantic query capability, let's now keeping ask about task decomposition, but this time, asking for its downside:


In [16]:
retrieved_docs = retriever.invoke("What are the downsides of Task Decomposition?")
for doc in retrieved_docs:
    print(doc)
    print('-'*80)

page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
--------------------------------------------------------------------------------
page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
--------------------------------------------------------------------------------
page_content='Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefit a lot from long or infinite context windows. Althou

Note that this time the most relevant results are different from the previous one.

Furthermore, the first results haven't the word ‘downside’, but related words like ‘challenges’, ‘limitation’ and ‘restricted’. This reinforces the capability of semantic search of embedding vectors in vector database.
.


# Contextualizing the question

In [17]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Chain with chat history

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Statefully manage chat history

In [19]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [20]:
ai_msg_1 = conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

Parent run bc4e900c-a8c2-40c2-9c1c-fe2bdc4e4be0 not found for run 279fea55-6881-45a7-a162-0f7821deb799. Treating as a root run.


In [21]:
ai_msg_1["answer"]

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves transforming big tasks into multiple manageable tasks to facilitate problem-solving and planning. This process can be done using prompting techniques like Chain of Thought or Tree of Thoughts, or with task-specific instructions.'

In [22]:
ai_msg_1["context"]

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a

In [23]:
ai_msg_2 = conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run b047c25e-6b7d-443e-8a63-becd8c55b03a not found for run 23e965e8-05f2-49b3-b0ad-7ff0893e5f1f. Treating as a root run.


In [24]:
ai_msg_2["answer"]

'Task decomposition can be achieved through various methods such as using prompting techniques like Chain of Thought or Tree of Thoughts, which guide models to break down tasks into smaller steps. Additionally, task-specific instructions can be provided to direct the decomposition process, such as asking for a story outline when writing a novel. Human inputs can also be utilized to assist in breaking down complex tasks into more manageable components.'

In [25]:
ai_msg_2["context"]

[Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree 